In [1]:
import bpy
blend_file_path = "group_network.blend"
bpy.ops.wm.open_mainfile(filepath=blend_file_path)

Read blend: "/Users/jan-hendrik/projects/ipyblender-gui/group_network.blend"


{'FINISHED'}

Info: Saved "group_network.blend"

Info: Saved "group_network.blend"

Info: Saved "group_network.blend"



Traceback (most recent call last):
  File "/Users/jan-hendrik/Library/Application Support/Blender/4.1/scripts/addons/molecularnodes/ui/ops.py", line 52, in poll
    return mol.mn['molecule_type'] in ['pdb', 'local']
           ~~~~~~^^^^^^^^^^^^^^^^^
KeyError: 'bpy_struct[key]: key "molecule_type" not found'
Traceback (most recent call last):
  File "/Users/jan-hendrik/Library/Application Support/Blender/4.1/scripts/addons/molecularnodes/ui/ops.py", line 52, in poll
    return mol.mn['molecule_type'] in ['pdb', 'local']
           ~~~~~~^^^^^^^^^^^^^^^^^
KeyError: 'bpy_struct[key]: key "molecule_type" not found'


In [3]:
import networkx as nx
import bpy

# Create the Tutte graph and calculate node positions
G = nx.tutte_graph()
node_positions = nx.spring_layout(G, dim=3, scale=2)
edges = list(G.edges)

def draw_network():

    # Function to delete all objects in a collection
    def clear_collection(collection):
        for obj in collection.objects:
            bpy.data.objects.remove(obj, do_unlink=True)
    
    # Function to get or create a collection
    def get_or_create_collection(name, parent=None):
        collection = bpy.data.collections.get(name)
        if not collection:
            collection = bpy.data.collections.new(name)
            if parent:
                parent.children.link(collection)
        else:
            clear_collection(collection)
        return collection
    
    # Get or create the "Graph" collection and its children "Nodes" and "Edges"
    graph_collection = get_or_create_collection("Graph", bpy.context.scene.collection)
    edges_collection = get_or_create_collection("Edges", graph_collection)
    nodes_collection = get_or_create_collection("Nodes", graph_collection)
    
    # Clear the Nodes collection
    clear_collection(nodes_collection)
    
    # Create spheres at the points and add them to the Nodes collection
    spheres = {}
    material = bpy.data.materials.get("Material1")
    
    for key, coords in node_positions.items():
        bpy.ops.mesh.primitive_uv_sphere_add(location=coords, radius=0.1)
        sphere = bpy.context.object
        sphere.name = f"my_point{key}"
        if material:
            sphere.data.materials.append(material)
        nodes_collection.objects.link(sphere)
        bpy.context.collection.objects.unlink(sphere)
        spheres[key] = sphere
    
    # Create plane objects at the center of each edge and add geometry nodes modifier
    for i, (edge1, edge2) in enumerate(edges):
        bpy.ops.mesh.primitive_plane_add(size=0.2)  
        plane = bpy.context.object
        plane.name = f"edge{i+1}"
        edges_collection.objects.link(plane)
        bpy.context.collection.objects.unlink(plane)
        
        node_group = bpy.data.node_groups.get("connector_goenode")
        modifier = plane.modifiers.new(name="Connect", type='NODES')
        modifier.node_group = node_group 
        
        # Set the Group input node to point to the corresponding spheres
        sphere1 = bpy.data.objects.get(f"my_point{edge1}")
        sphere2 = bpy.data.objects.get(f"my_point{edge2}")
        
        modifier["Socket_2"] = sphere1
        modifier["Socket_3"] = sphere2
        if material:
            modifier["Socket_4"] = material

# Call the function to draw the network
draw_network(scale=6)